In [ ]:
import os
import sys

repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/content/6D_pose"
branch = "pose_rgb"

if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url}")

if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

%cd 6D_pose

In [ ]:
from google.colab import drive
from utils.load_data import mount_drive

mount_drive()

dataset_root = "/content/drive/MyDrive/Linemod_preprocessed"
print(f"\n✅ Setup complete!")
print(f"📁 Dataset path: {dataset_root}")

In [ ]:
!pip install plyfile
from utils.projection_utils import *

setup_projection_utils(dataset_root)

In [ ]:
from src.pose_rgb.pointcloud_dataset import LineModPointCloudDataset
from src.pose_rgb.pointnet_model import PointNetPose
from utils.linemod_config import get_linemod_config
import torch
import numpy as np

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🔥 Using device: {DEVICE}")

# Load config
config = get_linemod_config(dataset_root)

In [ ]:
# Carica test dataset
test_dataset = LineModPointCloudDataset(
    root_dir=dataset_root,
    split='test',
    num_points=1024,
    use_rgb=True
)

print(f"Test samples: {len(test_dataset)}")

In [ ]:
# Carica modello PointNet salvato
model_path = "/content/drive/MyDrive/runs/pointnet_XXXXXXXX_XXXXXX/best_model.pth"  # MODIFICA QUI

checkpoint = torch.load(model_path, map_location=DEVICE)

# Initialize model
model = PointNetPose(input_channels=6, use_batch_norm=True).to(DEVICE)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Model loaded from: {model_path}")
print(f"   Epoch: {checkpoint['epoch']}")
print(f"   Val Loss: {checkpoint['val_loss']:.4f}")

## Oggetti Simmetrici LineMOD

LineMOD contiene alcuni oggetti con simmetria rotazionale:
- **10 (Eggbox)**: Simmetria rotazionale
- **11 (Glue)**: Simmetria rotazionale

Per questi oggetti usiamo **ADD-S** (Average Distance Symmetric) invece di ADD.

In [ ]:
# Definisci quali oggetti sono simmetrici
SYMMETRIC_OBJECTS = [10, 11]  # Eggbox e Glue

def is_symmetric(obj_id):
    """Controlla se un oggetto è simmetrico"""
    return obj_id in SYMMETRIC_OBJECTS

In [ ]:
from metrics.ADD_metric import compute_ADD_metric_quaternion

def compute_ADD_S_metric(model_points, gt_quat, gt_translation,
                         pred_quat, pred_translation):
    """
    Compute ADD-S metric per oggetti simmetrici.
    Per ogni punto trasformato con GT, trova il punto più vicino
    trasformato con la predizione.
    
    Args:
        model_points: (N, 3) array of 3D model points in metri
        gt_quat: (4,) ground truth quaternion
        gt_translation: (3,) ground truth translation in metri
        pred_quat: (4,) predicted quaternion
        pred_translation: (3,) predicted translation in metri
    
    Returns:
        add_s: Average closest point distance (ADD-S) metric
    """
    from src.pose_rgb.pose_utils import quaternion_to_rotation_matrix
    
    # Convert quaternions to rotation matrices
    R_gt = quaternion_to_rotation_matrix(gt_quat)
    R_pred = quaternion_to_rotation_matrix(pred_quat)
    
    # Transform model points
    gt_points = (R_gt @ model_points.T).T + gt_translation
    pred_points = (R_pred @ model_points.T).T + pred_translation
    
    # Per ogni punto GT, trova il punto predetto più vicino
    from scipy.spatial.distance import cdist
    distances = cdist(gt_points, pred_points, metric='euclidean')
    min_distances = distances.min(axis=1)
    
    add_s = min_distances.mean()
    return add_s

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm

def evaluate_sample(model, sample, config):
    """
    Valuta un singolo sample e calcola metriche.
    
    Returns:
        dict con metriche e predizioni
    """
    # Prepare input
    point_cloud = sample['point_cloud'].unsqueeze(0).to(DEVICE)
    bbox_info = sample['bbox_info'].unsqueeze(0).to(DEVICE)
    gt_rot = sample['rotation']
    gt_trans = sample['translation']
    obj_id = sample['object_id']
    
    # Inference
    with torch.no_grad():
        pred_rot, pred_trans = model(point_cloud, bbox_info)
    
    # Convert to numpy
    pred_rot = pred_rot.squeeze(0).cpu().numpy()
    pred_trans = pred_trans.squeeze(0).cpu().numpy()
    gt_rot = gt_rot.numpy()
    gt_trans = gt_trans.numpy()
    
    # Carica model 3D points
    model_3d = config.get_model_3d(obj_id, unit='m')
    
    # Calcola ADD o ADD-S
    if is_symmetric(obj_id):
        add_metric = compute_ADD_S_metric(
            model_3d, gt_rot, gt_trans, pred_rot, pred_trans
        )
        metric_name = "ADD-S"
    else:
        add_metric = compute_ADD_metric_quaternion(
            model_3d, gt_rot, gt_trans, pred_rot, pred_trans
        )
        metric_name = "ADD"
    
    # Calcola errore di translation
    trans_error = np.linalg.norm(gt_trans - pred_trans)
    
    # Calcola errore di rotation (angolo tra quaternioni)
    dot_product = np.abs(np.dot(pred_rot, gt_rot))
    dot_product = np.clip(dot_product, -1.0, 1.0)
    angle_error = 2 * np.arccos(dot_product) * 180 / np.pi
    
    return {
        'object_id': obj_id,
        'add_metric': add_metric,
        'metric_name': metric_name,
        'trans_error': trans_error,
        'angle_error': angle_error,
        'pred_rot': pred_rot,
        'pred_trans': pred_trans,
        'gt_rot': gt_rot,
        'gt_trans': gt_trans
    }

# Evalua alcuni sample
print("🔍 Evaluating samples...")
results = []

num_samples = 100  # Valuta 100 samples
for i in tqdm(range(num_samples)):
    sample = test_dataset[i]
    result = evaluate_sample(model, sample, config)
    results.append(result)

print(f"\n✅ Evaluated {len(results)} samples")

In [ ]:
# Statistiche per oggetto
import pandas as pd

df = pd.DataFrame(results)

# Group by object
stats_by_object = df.groupby('object_id').agg({
    'add_metric': ['mean', 'std', 'min', 'max'],
    'trans_error': ['mean', 'std'],
    'angle_error': ['mean', 'std']
}).round(4)

print("\n📊 Metrics by Object ID:")
print(stats_by_object)

# Overall statistics
print("\n📊 Overall Statistics:")
print(f"   Mean ADD/ADD-S: {df['add_metric'].mean():.4f} m")
print(f"   Mean Translation Error: {df['trans_error'].mean():.4f} m")
print(f"   Mean Rotation Error: {df['angle_error'].mean():.2f}°")

In [ ]:
# Visualizza distribuzione errori
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df['add_metric'], bins=30, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('ADD / ADD-S (m)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('ADD Metric Distribution')
axes[0].axvline(df['add_metric'].mean(), color='r', linestyle='--', 
                label=f'Mean: {df["add_metric"].mean():.3f}')
axes[0].legend()

axes[1].hist(df['trans_error'], bins=30, alpha=0.7, color='orange', edgecolor='black')
axes[1].set_xlabel('Translation Error (m)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Translation Error Distribution')
axes[1].axvline(df['trans_error'].mean(), color='r', linestyle='--',
                label=f'Mean: {df["trans_error"].mean():.3f}')
axes[1].legend()

axes[2].hist(df['angle_error'], bins=30, alpha=0.7, color='green', edgecolor='black')
axes[2].set_xlabel('Rotation Error (degrees)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Rotation Error Distribution')
axes[2].axvline(df['angle_error'].mean(), color='r', linestyle='--',
                label=f'Mean: {df["angle_error"].mean():.2f}°')
axes[2].legend()

plt.tight_layout()
plt.show()

## Visualizzazione Qualitativa

Visualizziamo alcune predizioni sovrapposte all'immagine RGB usando le utility di projection.

In [ ]:
import matplotlib.pyplot as plt

def visualize_inference(dataset, sample_idx, results_dict):
    """
    Visualizza la predizione vs GT usando projection utilities.
    """
    sample = dataset[sample_idx]
    result = results_dict
    
    # Carica immagine RGB originale
    obj_id = sample['object_id']
    img_id = sample['img_id']
    
    from pathlib import Path
    img_path = Path(dataset_root) / "data" / f"{obj_id:02d}" / "rgb" / f"{img_id:04d}.png"
    import cv2
    image = cv2.imread(str(img_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Camera intrinsics
    cam_K = sample['cam_K'].numpy()
    
    # Visualizza
    img_vis = visualize_pose_comparison(
        image, obj_id, cam_K,
        result['gt_rot'], result['gt_trans'],
        result['pred_rot'], result['pred_trans']
    )
    
    plt.figure(figsize=(12, 10))
    plt.imshow(img_vis)
    plt.axis('off')
    plt.title(f"Object {obj_id} | {result['metric_name']}: {result['add_metric']:.4f}m | "
              f"Trans Err: {result['trans_error']:.4f}m | Angle Err: {result['angle_error']:.2f}°")
    plt.show()

# Visualizza alcuni sample
print("🎨 Visualizing predictions...")
for i in [0, 10, 20, 30]:
    if i < len(results):
        print(f"\n--- Sample {i} ---")
        visualize_inference(test_dataset, i, results[i])

In [ ]:
# Trova i migliori e peggiori risultati
best_idx = df['add_metric'].idxmin()
worst_idx = df['add_metric'].idxmax()

print("\n🏆 Best Prediction:")
print(f"   Sample index: {best_idx}")
print(f"   ADD: {df.loc[best_idx, 'add_metric']:.4f} m")
visualize_inference(test_dataset, best_idx, results[best_idx])

print("\n❌ Worst Prediction:")
print(f"   Sample index: {worst_idx}")
print(f"   ADD: {df.loc[worst_idx, 'add_metric']:.4f} m")
visualize_inference(test_dataset, worst_idx, results[worst_idx])